In [ ]:
#단, best_model.h5, test_stage_1, train_stage_1은 현재 실행하고 있는 폴더 내에 복사해서 붙여넣기 필수!
#층화 추출법
# DB받아야하고
# 개인별로 몇개할지 정하고
# 코드수정할 부분 파악하고
# 텍스트 파일이 2개씩 생성되니까 그거 파악해야하고
# 주말까진 해야하고
import re
import Sentimental_Analyzer as SA
import Date_List as DL
import BSC_Crawler as crawler
from Hot_Topic_Analyzer import hot_topic_analyzer
from Aspect_Miner import aspect_miner
import pandas as pd
import pymysql
import random
from selenium import webdriver
import csv

# 무작위 10일을 선택.
select_date = random.sample(range(1,30),15) # 1부터 29 중에 10개를 중복없이 뽑는다. -> type : list
print("표본조사를 위한 날짜를 랜덤으로 선택합니다.")
print("선택된 날짜 : ", select_date)
print('---------------------------------------------------------------------------------------------------------------------------------')

driver = webdriver.Chrome("C:/chromedriver.exe")

conn=pymysql.connect(
    host='127.0.0.1',
    port=3306,
    user='root',
    passwd='3721',
    db='webnoveldb',
    charset='utf8'
)

curs=conn.cursor()
sql="select * from novellink where (idnovel >95 ) and (idnovel <= 500)"
#####################################id novel 1~99, 501~693, 5001~5031 완료########################################


curs.execute(sql)

rows=curs.fetchall()

novel_lists =[]
for row in rows:
    novel_lists.append(list(row)) #'작품ID', '작품제목', '조아라','문피아','카카오페이지','리디북스','시리즈온','네이버 웹소설']

#crawler.ig_login(driver)
crawler.ridi_login(driver)
crawler.naver_login(driver)

for novel_list in novel_lists:
    print('---------------------------------------------------------------------------------------------------------------------------------')
    print('\n')
    work_name=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', novel_list[1]).replace("  "," ")
    file_name='Crawling\\' + work_name +'.txt'
    print(work_name + "에 대한 작품 분석을 위한 크롤링을 시작합니다.")
    ####################인스타그램 크롤러################
    #instagram_list=[]
    #print(work_name + "에 대한 Instagram 크롤링을 진행하고 있습니다.")

    #instagram_list=crawler.ig_scraping(driver,novel_list[1])
    #print(work_name + "에 대한 Naver Blog 크롤링이 완료되었습니다.")

    #data = pd.DataFrame(instagram_list, columns = ['내용', '작성 날짜'])
    #data

    ####################디시인사이드 크롤러##################
    dcinside_list=[]
    try:
        print("- "+ work_name + "에 대한 Dcinside 크롤링을 진행하고 있습니다.")
        dcinside_list=crawler.dc_scraping(driver, novel_list[1])
        print("- "+ work_name + "에 대한 Dcinside 크롤링이 완료되었습니다.")
        #df = pd.DataFrame(dcinside_list, columns = ['작성 날짜', '제목', '내용 + 댓글'])
        #df
    except:
        pass

    ####################인스티즈 크롤러##############
    #instiz_list_list=[]
    #print(work_name + "에 대한 Instiz 크롤링을 진행하고 있습니다.")

    #instiz_list=crawler.dc_scraping(driver, novel_list[985][1])
    #print(work_name + "에 대한 Naver Blog 크롤링이 완료되었습니다.")


    ####################네이버 크롤러################
    naver_list=[]
    print("- "+ work_name + "에 대한 Naver Blog 크롤링을 진행하고 있습니다.")

    naver_list=crawler.naver_crawling(novel_list[1])
    print("- "+ work_name + "에 대한 Naver Blog 크롤링이 완료되었습니다.")
    ####################다음 크롤러##################
    daum_list=[]
    print("- "+ work_name + "에 대한 Daum Blog 크롤링을 진행하고 있습니다.")

    try:
        daum_list=crawler.daum_crawling(novel_list[1])
        print("- "+ work_name + "에 대한 Daum Blog 크롤링이 완료되었습니다.")

    except:
        pass
    ####################티스토리 크롤러##############
    tistory_list=[]
    try:
        print("- "+ work_name + "에 대한 Tistory Blog 크롤링을 진행하고 있습니다.")

        tistory_list=crawler.tistory_crawling(novel_list[1])
        print("- "+ work_name + "에 대한 Tistory Blog 크롤링이 완료되었습니다.")

    except:
        pass
    #[title, author, date, post]

    ############크롤러 결과중 내용부분만 textfile화############

    f=open(file_name, 'a',encoding="utf-8")
    for content in dcinside_list:
        content[2]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[2])
        f.write("%s\n" % content[2])
    for content in dcinside_list:
        content[2]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[2])
        f.write("%s\n" % content[2])
    for content in naver_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    for content in naver_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    for content in daum_list:
        content[3]=re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,]', '', content[3])
        f.write("%s\n" % content[3])
    f.close()
    #f=open('Crawling\\{}.txt'.format(re.sub('[^ ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9\.\,], '', novel_list[1])), 'a',encoding="utf-8")
    #with open('{}TR.txt'.format(title), 'a',encoding="utf-8") as f:
    #for content in instagram_list:
    #    content[0]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9.]', '', content[1])
    #    f.write("%s\n" % content[1])

    #for content in instiz_list:
    #    content[3]=re.sub('[^ ㄱ-ㅎ|ㅏ-ㅣ|가-힣a-zA-Z0-9\:\/\-\+\*\~\!\`\'\"\!\@\#\$\%\^\&\*\(\.\,\<\/\?\:\;\[\]\{\}\\\|]', '', content[3])
    #    f.write("%s\n" % content[3])
    
    ############Textrank 분석############
    topic_list=[]
    print('\n')
    print('-------------------------------------------------------------------------------------------------')
    print('\n')
    print(work_name + "에 대한 TextRank 분석을 시작합니다.")
    topic_list=hot_topic_analyzer(work_name)
    if(topic_list==None):
        topic_list=[]
        for i in range(0,5):
            topic_list.append('NULL')
    print("- "+ work_name + "에 대한 TextRank 분석결과 ")
    print(topic_list)
    
    
    ###########Aspect mining 분석########\
    character_word_list=[]
    story_word_list=[]
    mood_word_list=[]
    print('\n')
    print('-------------------------------------------------------------------------------------------------')
    print('\n')
    print(work_name + "에 대한 Aspect Mining 분석을 시작합니다.")
    character_word_list, story_word_list, mood_word_list = aspect_miner(work_name)
    print("- "+ work_name + "에 대한 Aspect Mining 분석결과 (주인공, 스토리, 분위기 키워드순)")
    print(character_word_list)
    print(story_word_list)
    print(mood_word_list)
    """
    try:
        character_word_list, story_word_list, mood_word_list = aspect_miner(novel_list[1])

    except:
        for i in range(0,5):
            character_word_list.append('NULL')
            story_word_list.append('NULL')
            mood_word_list.append('NULL')
    """
    f_w = open('TA_analysis_result.csv', 'a', encoding='cp949', newline='')
    wr = csv.writer(f_w)
    result=topic_list+character_word_list+story_word_list+mood_word_list
    result.insert(0,novel_list[0])
    wr.writerow(result)
    f_w.close()

    
    #########################################################감성분석#########################################################

    date_list=DL.make_date_list()
    print('\n')
    print('-------------------------------------------------------------------------------------------------')
    print('\n')
    print(work_name + "에 대한 Sentimental Analysis 분석을 시작합니다.")
    
    if(novel_list[2]):
        print("- "+ work_name + "에 대한 Joara의 독자 리뷰 크롤링을 진행하고 있습니다.")
        joara_comments=crawler.joara_comments(driver,novel_list[2])
        print("- "+ work_name + "에 대한 Joara의 독자 리뷰 크롤링이 완료되었습니다.")

        #data = pd.DataFrame(joara_comments, columns = ['내용', '작성 날짜'])
        #print(data)
        for review in joara_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1

    if(novel_list[5]):
        print("- "+ work_name + "에 대한 Ridibooks의 독자 리뷰 크롤링을 진행하고 있습니다.")
        ridi_comments=crawler.ridi_comments(select_date,driver,novel_list[5])
        print("- "+ work_name + "에 대한 Ridibooks의 독자 리뷰 크롤링이 완료되었습니다.")

        #data = pd.DataFrame(ridi_comments, columns = ['내용', '작성 날짜'])
        #print(data)
        for review in ridi_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1

    if(novel_list[6]):
        print("- "+ work_name + "에 대한 Naver Series의 독자 리뷰 크롤링을 진행하고 있습니다.")

        serieson_comments=crawler.serieson_comments(select_date,driver,novel_list[6])
        print("- "+ work_name + "에 대한 Naver Serieson의 독자 리뷰 크롤링이 완료되었습니다.")

        #data = pd.DataFrame(serieson_comments, columns = ['내용', '작성 날짜'])
        #data
        for review in serieson_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1
    
    if(novel_list[7]):
        print("- "+ work_name + "에 대한 Naver 웹소설의 독자 리뷰 크롤링을 진행하고 있습니다.")

        naver_comments=crawler.naver_comments(driver,novel_list[7])
        print("- "+ work_name + "에 대한 Naver 웹소설의 독자 리뷰 크롤링이 완료되었습니다.")

        #data = pd.DataFrame(naver_comments, columns = ['내용', '작성 날짜'])
        #print(data) <-> data
        for review in naver_comments:
            score=float(SA.predict(review[1]))
            pos=DL.date_list_pos(review[0])
            date_list[pos][1]=date_list[pos][1] + score
            date_list[pos][2]+=1
    


    f_w = open('S_analysis_result.csv', 'a', encoding='cp949', newline='')
    wr = csv.writer(f_w)
    result4showing=[]
    for date in date_list:
        result=[]
        result.append(date[0])
        try:
            result.append("%.2f" % (date[1]/date[2]))
            result4showing.append("%.2f" % (date[1]/date[2]))
        except:
            result.append('0')
            result4showing.append('0')
            
        result.insert(0,novel_list[0])
        wr.writerow(result)
    f_w.close()
    
    print('\n')
    print("- "+ work_name + "에 대한 Sentimental 분석결과 ( 2020.01 ~ 2020.10 ) ")
    print(result4showing)
    print('\n')
    print(work_name + "에 대한 작품 분석을 위한 크롤링이 모두 완료되었습니다.")
    print('\n')
    print('---------------------------------------------------------------------------------------------------------------------------------')
    

표본조사를 위한 날짜를 랜덤으로 선택합니다.
선택된 날짜 :  [21, 24, 29, 19, 23, 25, 12, 6, 8, 17, 22, 13, 2, 26, 28]
---------------------------------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------------------------------


약빨이 신선함에 대한 작품 분석을 위한 크롤링을 시작합니다.
- 약빨이 신선함에 대한 Dcinside 크롤링을 진행하고 있습니다.
- 약빨이 신선함에 대한 Dcinside 크롤링이 완료되었습니다.
- 약빨이 신선함에 대한 Naver Blog 크롤링을 진행하고 있습니다.
- 약빨이 신선함에 대한 Naver Blog 크롤링이 완료되었습니다.
- 약빨이 신선함에 대한 Daum Blog 크롤링을 진행하고 있습니다.
- 약빨이 신선함에 대한 Daum Blog 크롤링이 완료되었습니다.
- 약빨이 신선함에 대한 Tistory Blog 크롤링을 진행하고 있습니다.
- 약빨이 신선함에 대한 Tistory Blog 크롤링이 완료되었습니다.


-------------------------------------------------------------------------------------------------


약빨이 신선함에 대한 TextRank 분석을 시작합니다.
- 약빨이 신선함에 대한 TextRank 분석결과 
['한 ', '유화 부인 ', '금 손 ', '담 덕 ', '동해 용왕 ']


-------------------------------------------------------------------------------------